In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def part2q1(f, h):
    m,n = f.shape
    h = 1/(n-1)
    x = np.linspace(0,1,n)
    y = np.linspace(0,1,m)
    df = np.zeros_like(f) #modify as needed
    d2f = np.zeros_like(f) #modify as needed

    #Add code here

    # Build A as a sparse pentadiagonal matrix
    size = 2 * n
    lower2 = np.zeros(size)
    lower1 = np.zeros(size)
    main = np.zeros(size)
    upper1 = np.zeros(size)
    upper2 = np.zeros(size)

    for i in range(size):
        # Even index: u'
        if i % 2 == 0:
            if i == 0:
                main[i] = 1
                upper1[i+1] = 2
                upper2[i+2] = -h
            elif i == size - 2:
                main[i] = 1
                lower1[i-1] = 2
                lower2[i-2] = h
            else:
                main[i] = 16
                upper1[i+1] = 7
                lower1[i-1] = 7
                upper2[i+2] = -h
                lower2[i-2] = h
        # Odd index: u''
        else:
            if i == 1:
                main[i] = h
                upper1[i+1] = -6
                upper2[i+2] = 5 * h
            elif i == size - 1:
                main[i] = -h
                lower1[i-1] = -6
                lower2[i-2] = -5 * h
            else:
                main[i] = 8 * h
                upper1[i+1] = 9
                lower1[i-1] = -9
                upper2[i+2] = -h
                lower2[i-2] = -h

    from scipy.sparse import dia_matrix
    A = dia_matrix(([lower2, lower1, main, upper1, upper2], [-2, -1, -0, 1, 2]), shape=(size, size)).tocsr()

    # Constants for the vector b
    c31, c22 = 15 / h, 24 / h
    cb11, cb21, cb31 = -3.5 / h, 4 / h, -0.5 / h
    cb12, cb22, cb32 = 9 / h, -12 / h, 3 / h

    # For each row in f, compute df and d2f
    from scipy.sparse.linalg import spsolve
    for i in range(m):
        b = np.zeros(size)
        f_row = f[i, :]
        for j in range(n):
            if j == 0:
                b[0] = cb11 * f_row[0] + cb21 * f_row[1] + cb31 * f_row[2]
                b[1] = cb12 * f_row[0] + cb22 * f_row[1] + cb32 * f_row[2]
            elif j == n - 1:
                b[-2] = -(cb11 * f_row[-1] + cb21 * f_row[-2] + cb31 * f_row[-3])
                b[-1] = -(cb12 * f_row[-1] + cb22 * f_row[-2] + cb32 * f_row[-3])
            else:
                idx = 2 * j
                b[idx] = c31 * (f_row[j+1] - f_row[j-1])
                b[idx+1] = c22 * (f_row[j-1] - 2 * f_row[j] + f_row[j+1])

        # Solve the system
        out = spsolve(A, b)
        df[i, :] = out[::2]
        d2f[i, :] = out[1::2]

    return df, d2f

In [ ]:
def dualfd1(f):
    """
    Code implementing implicit finite difference scheme for special case m=1
    Implementation is not efficient.
    Input:
        f: n-element numpy array
    Output:
        df, d2f: computed 1st and 2nd derivatives
    """
    #parameters, grid
    n = f.size
    h = 1/(n-1)
    x = np.linspace(0,1,n)
    
    #fd method coefficients
    #interior points:
    L1 = [7,h,16,0,7,-h]
    L2 = [-9,-h,0,8*h,9,-h]
    
    #boundary points:
    L1b = [1,0,2,-h]
    L2b = [0,h,-6,5*h]

    L1b2 = [2,h,1,0]
    L2b2 = [-6,-5*h,0,-h]

    A = np.zeros((2*n,2*n))
    #iterate filling a row of A each iteration
    for i in range(n):
        #rows 0 and N-1
        if i==0:
            #Set boundary eqn 1
            A[0,0:4] = L1b
            #Set boundary eqn 2
            A[1,0:4] = L2b
        elif i==n-1:
            A[-2,-4:] = L1b2
            A[-1,-4:] = L2b2
        else:
            #interior rows
            #set equation 1
            ind = 2*i
            A[ind,ind-2:ind+4] = L1
            #set equation 2
            A[ind+1,ind-2:ind+4] = L2

    #set up RHS
    b = np.zeros(2*n)
    c31,c22,cb11,cb21,cb31,cb12,cb22,cb32 = 15/h,24/h,-3.5/h,4/h,-0.5/h,9/h,-12/h,3/h
    for i in range(n):
        if i==0:
            b[i] = cb11*f[0]+cb21*f[1]+cb31*f[2]
            b[i+1] = cb12*f[0]+cb22*f[1]+cb32*f[2]
        elif i==n-1:
            b[-2] =-(cb11*f[-1]+cb21*f[-2]+cb31*f[-3])
            b[-1] = -(cb12*f[-1]+cb22*f[-2]+cb32*f[-3])
        else:
            ind = 2*i
            b[ind] = c31*(f[i+1]-f[i-1])
            b[ind+1] = c22*(f[i-1]-2*f[i]+f[i+1])
    out = np.linalg.solve(A,b)
    df = out[::2]
    d2f = out[1::2]
    return df,d2f



In [ ]:
def fd2(f):
    """
    Computes the first and second derivatives with respect to x using second-order finite difference methods.
    
    Input:
    f: m x n array whose 1st and 2nd derivatives will be computed with respect to x
    
    Output:
     df, d2f: m x n arrays containing 1st and 2nd derivatives of f with respect to x
    """

    m,n = f.shape
    h = 1/(n-1)
    df = np.zeros_like(f) 
    d2f = np.zeros_like(f)
    
    # First derivative 
    # Centered differences for the interior 
    df[:, 1:-1] = (f[:, 2:] - f[:, :-2]) / (2 * h)

    # One-sided differences at the boundaries
    df[:, 0] = (-3 * f[:, 0] + 4 * f[:, 1] - f[:, 2]) / (2 * h)
    df[:, -1] = (3 * f[:, -1] - 4 * f[:, -2] + f[:, -3]) / (2 * h)
    
    # Second derivative 
    # Centered differences for the interior 
    d2f[:, 1:-1] = (f[:, 2:] - 2 * f[:, 1:-1] + f[:, :-2]) / (h**2)
    
    # One-sided differences at the boundaries
    d2f[:, 0] = (2 * f[:, 0] - 5 * f[:, 1] + 4 * f[:, 2] - f[:, 3]) / (h**2)
    d2f[:, -1] = (2 * f[:, -1] - 5 * f[:, -2] + 4 * f[:, -3] - f[:, -4]) / (h**2)
    
    return df, d2f

In [ ]:
def part2q2():
    from time import time

    # Initial variables
    n_values = list(range(10, 1001, 10)) # Use n = 10, 20, ..., 1000
    error_fd2 = []
    error_part2q1 = []
    runtime_fd2 = []
    runtime_part2q1 = []

    for n in n_values:
        # Generate f and df
        m = n  # Since m comparable to n
        x = np.linspace(0, 1, n)
        y = np.linspace(0, 1, m)
        X, Y = np.meshgrid(x, y)
        f = np.sin(X) * Y
        df = np.cos(X) * Y  # Exact derivative

        # Run fd2, store runtime and error
        start_time = time()
        df_fd2, _ = fd2(f)
        runtime_fd2.append(time() - start_time)
        error_fd2.append(np.max(df_fd2[:, 1:-1] - df[:, 1:-1]))

        # Run part2q1, store runtime and error
        start_time = time()
        df_part2q1, _ = part2q1(f, 1/(n-1))
        runtime_part2q1.append(time() - start_time)
        error_part2q1.append(np.max(df_part2q1[:, 1:-1] - df[:, 1:-1]))

    # Plot accuracy results
    plt.plot(n_values, np.array(error_part2q1), label="part2q1")
    plt.plot(n_values, np.array(error_fd2), label="fd2")
    plt.xlabel("n")
    plt.ylabel("Error")
    plt.title("Accuracy Comparison (First Derivative)")
    plt.legend()
    plt.show()

    # Plot runtime results
    plt.plot(n_values, np.array(runtime_part2q1), label="part2q1")
    plt.plot(n_values, np.array(runtime_fd2), label="fd2")
    plt.xlabel("n")
    plt.ylabel("Runtime")
    plt.title("Runtime Comparison")
    plt.legend()
    plt.show()

In [ ]:
part2q2()